In [1]:
import gym
import numpy as np
from gym.wrappers import AtariPreprocessing
gym.new_step_api=True
env = gym.make('Pong-v0')
H = 1600 # number of hidden layer neurons
D = 80 * 80 # input dimensionality: 80x80 grid
model = {}
model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
model['W2'] = np.random.randn(H) / np.sqrt(H)
# hyperparameters
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  I=np.asarray(I)
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  
  I[I == 144] = 0 # erase background (background type 1)
  
  I[I == 109] = 0 # erase background (background type 2)
  
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(float).ravel()

def discount_rewards(r):
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(epx, eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}

def model_step(model, observation, prev_x):

  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x
  
  # forward the policy network and sample an action from the returned probability
  aprob, _ = policy_forward(x)
  action = 2 if aprob >= 0.5 else 3 # roll the dice!
  
  return action, prev_x

def play_game(env, model):
  observation = env.reset()

  frames = []
  cumulated_reward = 0

  prev_x = None # used in computing the difference frame

  for t in range(1000):
      frames.append(env.render(mode = 'rgb_array'))
      action, prev_x = model_step(model, observation, prev_x)
      observation, reward, done, info = env.step(action)
      cumulated_reward += reward
      if done:
          print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
          break
  print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))
  env.close()
  display_frames_as_gif(frames)

def train_model(env, model, total_episodes = 100):
  hist = []
  observation = env.reset()

  prev_x = None # used in computing the difference frame
  xs,hs,dlogps,drs = [],[],[],[]
  running_reward = None
  reward_sum = 0
  episode_number = 0

  while True:
  
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
      episode_number += 1

      # stack together all inputs, hidden states, action gradients, and rewards for this episode
      epx = np.vstack(xs)
      eph = np.vstack(hs)
      epdlogp = np.vstack(dlogps)
      epr = np.vstack(drs)
      xs,hs,dlogps,drs = [],[],[],[] # reset array memory

      # compute the discounted reward backwards through time
      discounted_epr = discount_rewards(epr)
      # standardize the rewards to be unit normal (helps control the gradient estimator variance)
      discounted_epr -= np.mean(discounted_epr)
      discounted_epr /= np.std(discounted_epr)

      epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
      grad = policy_backward(epx, eph, epdlogp)
      for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

      # perform rmsprop parameter update every batch_size episodes
      if episode_number % batch_size == 0:
        for k,v in model.items():
          g = grad_buffer[k] # gradient
          rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
          model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
          grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

      # boring book-keeping
      running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
      hist.append((episode_number, reward_sum, running_reward))
      print ('episode %f, reward total was %f. running mean: %f' % (episode_number, reward_sum, running_reward))
      reward_sum = 0
      observation = env.reset() # reset env
      prev_x = None
      if episode_number == total_episodes: 
        return hist

   
    

C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\envs\registration.py:592: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [2]:
%time hist1 = train_model(env, model, total_episodes=7000)

C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


episode 1.000000, reward total was -21.000000. running mean: -21.000000
episode 2.000000, reward total was -21.000000. running mean: -21.000000
episode 3.000000, reward total was -20.000000. running mean: -20.990000
episode 4.000000, reward total was -21.000000. running mean: -20.990100
episode 5.000000, reward total was -20.000000. running mean: -20.980199
episode 6.000000, reward total was -21.000000. running mean: -20.980397
episode 7.000000, reward total was -21.000000. running mean: -20.980593
episode 8.000000, reward total was -21.000000. running mean: -20.980787
episode 9.000000, reward total was -20.000000. running mean: -20.970979
episode 10.000000, reward total was -21.000000. running mean: -20.971269
episode 11.000000, reward total was -19.000000. running mean: -20.951557
episode 12.000000, reward total was -21.000000. running mean: -20.952041
episode 13.000000, reward total was -21.000000. running mean: -20.952521
episode 14.000000, reward total was -20.000000. running mean

episode 114.000000, reward total was -21.000000. running mean: -20.550743
episode 115.000000, reward total was -21.000000. running mean: -20.555236
episode 116.000000, reward total was -20.000000. running mean: -20.549684
episode 117.000000, reward total was -21.000000. running mean: -20.554187
episode 118.000000, reward total was -20.000000. running mean: -20.548645
episode 119.000000, reward total was -21.000000. running mean: -20.553158
episode 120.000000, reward total was -19.000000. running mean: -20.537627
episode 121.000000, reward total was -21.000000. running mean: -20.542251
episode 122.000000, reward total was -21.000000. running mean: -20.546828
episode 123.000000, reward total was -21.000000. running mean: -20.551360
episode 124.000000, reward total was -21.000000. running mean: -20.555846
episode 125.000000, reward total was -21.000000. running mean: -20.560288
episode 126.000000, reward total was -20.000000. running mean: -20.554685
episode 127.000000, reward total was -

episode 225.000000, reward total was -20.000000. running mean: -20.402876
episode 226.000000, reward total was -21.000000. running mean: -20.408847
episode 227.000000, reward total was -21.000000. running mean: -20.414758
episode 228.000000, reward total was -20.000000. running mean: -20.410611
episode 229.000000, reward total was -19.000000. running mean: -20.396505
episode 230.000000, reward total was -21.000000. running mean: -20.402540
episode 231.000000, reward total was -20.000000. running mean: -20.398514
episode 232.000000, reward total was -21.000000. running mean: -20.404529
episode 233.000000, reward total was -20.000000. running mean: -20.400484
episode 234.000000, reward total was -21.000000. running mean: -20.406479
episode 235.000000, reward total was -19.000000. running mean: -20.392414
episode 236.000000, reward total was -19.000000. running mean: -20.378490
episode 237.000000, reward total was -20.000000. running mean: -20.374705
episode 238.000000, reward total was -

episode 336.000000, reward total was -19.000000. running mean: -20.384421
episode 337.000000, reward total was -21.000000. running mean: -20.390577
episode 338.000000, reward total was -21.000000. running mean: -20.396671
episode 339.000000, reward total was -21.000000. running mean: -20.402705
episode 340.000000, reward total was -19.000000. running mean: -20.388678
episode 341.000000, reward total was -20.000000. running mean: -20.384791
episode 342.000000, reward total was -20.000000. running mean: -20.380943
episode 343.000000, reward total was -19.000000. running mean: -20.367134
episode 344.000000, reward total was -20.000000. running mean: -20.363462
episode 345.000000, reward total was -19.000000. running mean: -20.349828
episode 346.000000, reward total was -19.000000. running mean: -20.336329
episode 347.000000, reward total was -21.000000. running mean: -20.342966
episode 348.000000, reward total was -21.000000. running mean: -20.349536
episode 349.000000, reward total was -

...

episode 6502.000000, reward total was -12.000000. running mean: -12.864112
episode 6503.000000, reward total was -17.000000. running mean: -12.905471
episode 6504.000000, reward total was -6.000000. running mean: -12.836416
episode 6505.000000, reward total was -17.000000. running mean: -12.878052
episode 6506.000000, reward total was -15.000000. running mean: -12.899271
episode 6507.000000, reward total was -17.000000. running mean: -12.940278
episode 6508.000000, reward total was -15.000000. running mean: -12.960876
episode 6509.000000, reward total was -16.000000. running mean: -12.991267
episode 6510.000000, reward total was -15.000000. running mean: -13.011354
episode 6511.000000, reward total was -14.000000. running mean: -13.021241
episode 6512.000000, reward total was -5.000000. running mean: -12.941028
episode 6513.000000, reward total was -14.000000. running mean: -12.951618
episode 6514.000000, reward total was -13.000000. running mean: -12.952102
episode 6515.000000, reward

episode 6612.000000, reward total was -18.000000. running mean: -13.007449
episode 6613.000000, reward total was -17.000000. running mean: -13.047374
episode 6614.000000, reward total was -15.000000. running mean: -13.066901
episode 6615.000000, reward total was -14.000000. running mean: -13.076232
episode 6616.000000, reward total was -14.000000. running mean: -13.085469
episode 6617.000000, reward total was -15.000000. running mean: -13.104615
episode 6618.000000, reward total was -10.000000. running mean: -13.073568
episode 6619.000000, reward total was -10.000000. running mean: -13.042833
episode 6620.000000, reward total was -13.000000. running mean: -13.042404
episode 6621.000000, reward total was -14.000000. running mean: -13.051980
episode 6622.000000, reward total was -5.000000. running mean: -12.971461
episode 6623.000000, reward total was -8.000000. running mean: -12.921746
episode 6624.000000, reward total was -13.000000. running mean: -12.922529
episode 6625.000000, reward

episode 6722.000000, reward total was -9.000000. running mean: -12.547861
episode 6723.000000, reward total was -9.000000. running mean: -12.512382
episode 6724.000000, reward total was -12.000000. running mean: -12.507258
episode 6725.000000, reward total was -11.000000. running mean: -12.492186
episode 6726.000000, reward total was -16.000000. running mean: -12.527264
episode 6727.000000, reward total was -14.000000. running mean: -12.541991
episode 6728.000000, reward total was -17.000000. running mean: -12.586571
episode 6729.000000, reward total was -15.000000. running mean: -12.610706
episode 6730.000000, reward total was -13.000000. running mean: -12.614599
episode 6731.000000, reward total was -9.000000. running mean: -12.578453
episode 6732.000000, reward total was -9.000000. running mean: -12.542668
episode 6733.000000, reward total was -2.000000. running mean: -12.437241
episode 6734.000000, reward total was -15.000000. running mean: -12.462869
episode 6735.000000, reward to

episode 6832.000000, reward total was -14.000000. running mean: -12.505032
episode 6833.000000, reward total was -1.000000. running mean: -12.389982
episode 6834.000000, reward total was -16.000000. running mean: -12.426082
episode 6835.000000, reward total was -16.000000. running mean: -12.461821
episode 6836.000000, reward total was -18.000000. running mean: -12.517203
episode 6837.000000, reward total was -9.000000. running mean: -12.482031
episode 6838.000000, reward total was -17.000000. running mean: -12.527210
episode 6839.000000, reward total was -17.000000. running mean: -12.571938
episode 6840.000000, reward total was -19.000000. running mean: -12.636219
episode 6841.000000, reward total was -14.000000. running mean: -12.649857
episode 6842.000000, reward total was -6.000000. running mean: -12.583358
episode 6843.000000, reward total was -13.000000. running mean: -12.587525
episode 6844.000000, reward total was -16.000000. running mean: -12.621649
episode 6845.000000, reward 

episode 6942.000000, reward total was -12.000000. running mean: -12.381538
episode 6943.000000, reward total was -13.000000. running mean: -12.387723
episode 6944.000000, reward total was -16.000000. running mean: -12.423845
episode 6945.000000, reward total was -15.000000. running mean: -12.449607
episode 6946.000000, reward total was -12.000000. running mean: -12.445111
episode 6947.000000, reward total was -13.000000. running mean: -12.450660
episode 6948.000000, reward total was -9.000000. running mean: -12.416153
episode 6949.000000, reward total was -17.000000. running mean: -12.461992
episode 6950.000000, reward total was -7.000000. running mean: -12.407372
episode 6951.000000, reward total was -16.000000. running mean: -12.443298
episode 6952.000000, reward total was -13.000000. running mean: -12.448865
episode 6953.000000, reward total was -9.000000. running mean: -12.414376
episode 6954.000000, reward total was -11.000000. running mean: -12.400233
episode 6955.000000, reward 